# Import Package

In [1]:
import torch
import torchinfo
from torchvision.models import resnet50
from torch.utils.data import DataLoader

C:\Users\Prolay\anaconda3\envs\torch_2.1.0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from level_1_dataloader import image_dataloader
from model.level_1_classifier import level_1_classifier, level_1_model, level_1_output_layer
from model.level_2_classifier import level_2_classifier, level_2_output_layer, level_2_pre_model_concate, level_2_post_model
from Trainer_1 import Trainer_level_1, Trainer_level_2
from Tester_1 import Tester_level_1, Tester_level_2

import copy

In [1]:
import torch
import torchinfo
from torchvision.models import resnet50
from torch.utils.data import DataLoader

from level_1_dataloader import image_dataloader
from model.level_1_classifier import level_1_classifier, level_1_model, level_1_output_layer
from model.level_2_classifier import level_2_classifier, level_2_output_layer, level_2_pre_model_concate, level_2_post_model
from Trainer_1 import Trainer_level_1, Trainer_level_2
from Tester_1 import Tester_level_1, Tester_level_2
from jbu_module import JointBilateralUpsampling  # Ensure you import the JBU module

import copy
import os

C:\Users\Prolay\anaconda3\envs\torch_2.1.0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset load (Level 1)

In [2]:
train_set = image_dataloader(csv_file = 'train_set.csv', root_dir = 'coverpage/coverpage/')
val_set = image_dataloader(csv_file = 'val_set.csv', root_dir = 'coverpage/coverpage/')
test_set = image_dataloader(csv_file = 'test_set.csv', root_dir = 'coverpage/coverpage/')

In [3]:
print('no. of training sample',len(train_set))
print('no. of validation sample',len(val_set))
print('no. of testing sample',len(test_set))

no. of training sample 22340
no. of validation sample 2804
no. of testing sample 2809


In [4]:

train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
print(len(train_dataloader))

val_dataloader = DataLoader(val_set, batch_size=32, shuffle=True)
print(len(val_dataloader))

test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)
print(len(test_dataloader))

699
88
88


# Assign GPU for model

In [5]:
def assign_gpu_model(model):
    print("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    return model

# Build Feature Extraction/ fine tune Model

In [6]:


# Fine-tune and load model
def fine_tune_load_model():
    model = resnet50(weights="IMAGENET1K_V2")
    print(model.modules)
    print(model.fc.in_features)
    
    modules = list(model.children())[:-1]
    new_model = torch.nn.Sequential(*modules)
    new_model.fc = torch.nn.Flatten()
    return new_model


    

In [7]:
FE_model = fine_tune_load_model()

<bound method Module.modules of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256,

In [8]:
used_model_feature_size = 2048  
feature_size_extract_from_level_1 = 64


In [9]:
torchinfo.summary(FE_model, (3, 224, 224), batch_dim=0, col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose=1)


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Sequential                               [1, 3, 224, 224]          [1, 2048]                 --                        --                        --
├─Conv2d: 1-1                            [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     [7, 7]                    118,013,952
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         [1, 64, 112, 112]         128                       --                        128
├─ReLU: 1-3                              [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --                        --
├─MaxPool2d: 1-4                         [1, 64, 112, 112]         [1, 64, 56, 56]           --                        3                         --
├─Sequential: 1-5                        [1, 64, 56, 56]           [1, 256, 56, 56]          --

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Sequential                               [1, 3, 224, 224]          [1, 2048]                 --                        --                        --
├─Conv2d: 1-1                            [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     [7, 7]                    118,013,952
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         [1, 64, 112, 112]         128                       --                        128
├─ReLU: 1-3                              [1, 64, 112, 112]         [1, 64, 112, 112]         --                        --                        --
├─MaxPool2d: 1-4                         [1, 64, 112, 112]         [1, 64, 56, 56]           --                        3                         --
├─Sequential: 1-5                        [1, 64, 56, 56]           [1, 256, 56, 56]          --

In [ ]:
import os

# Level 1 Model Parameters

In [10]:

# Directory and training parameters
used_model = 'ResNet-50'
used_model_feature_size = 2048  
level_1_checkpoint_dir = './checkpoints/' + used_model + '/level_1/'
level_1_model_file = 'model.pth'
level_1_csvlogger_file = 'log.csv'
level_1_weights_path = os.path.join(level_1_checkpoint_dir, level_1_model_file)
feature_size_extract_from_level_1 = 64
no_epoch = 10
lr = 0.0001
optimizer = 'AdamW'

# Create new checkpoint directory for level 1
new_checkpoint_dir = 'Downloads/checkpoints'
level_1_checkpoint_dir = os.path.join(new_checkpoint_dir, used_model, 'level_1')
os.makedirs(level_1_checkpoint_dir, exist_ok=True)
level_1_weights_path = os.path.join(level_1_checkpoint_dir, level_1_model_file)

In [11]:
classifier_level_1_with_jbu = level_1_classifier(feature_size=used_model_feature_size, feature_size_extract_from_level_1=feature_size_extract_from_level_1)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1=feature_size_extract_from_level_1, no_class=2)
model_level_1 = level_1_model(FE_model, classifier_level_1_with_jbu, output_layer_level_1)

# Build Level 1 Model

In [ ]:
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size,feature_size_extract_from_level_1=feature_size_extract_from_level_1, model=None)
# level_1_model = assign_gpu_model(level_1_model)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1 = feature_size_extract_from_level_1, no_class = 2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)

In [12]:
classifier_level_1_with_jbu

level_1_classifier(
  (activation): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (linear1): Linear(in_features=2048, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (linear3): Linear(in_features=256, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=64, bias=True)
  (jbu): JointBilateralUpsampling()
)

In [ ]:
output_layer_level_1

In [13]:
# Create dummy inputs for summary
dummy_input = torch.randn(1, 3, 224, 224)  # Batch size of 1 for simplicity
dummy_guidance = torch.randn(1, 3, 224, 224)  # Same size as the input image

# Use torchinfo to summarize the model with both inputs
torchinfo.summary(model_level_1, input_data=(dummy_input, dummy_guidance), col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose=1)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [14]:
# Define the new classifier with JBU
classifier_level_1_with_jbu = level_1_classifier(feature_size=used_model_feature_size, feature_size_extract_from_level_1=feature_size_extract_from_level_1)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1=feature_size_extract_from_level_1, no_class=2)
model_level_1 = level_1_model(FE_model, classifier_level_1_with_jbu, output_layer_level_1)


In [15]:
# Move model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_level_1.to(device)


level_1_model(
  (pre_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Con

In [16]:
# Create dummy inputs for summary and move them to the same device as the model
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # Batch size of 1 for simplicity
dummy_guidance = torch.randn(1, 3, 224, 224).to(device)  # Same size as the input image

In [17]:
torchinfo.summary(model_level_1, input_data=(dummy_input, dummy_guidance), col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose=1)


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, Conv2d: 2, BatchNorm2d: 2, ReLU: 2, MaxPool2d: 2, Sequential: 2, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, Sequential: 4, Conv2d: 5, BatchNorm2d: 5, ReLU: 4, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Sequential: 2, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, Sequential: 4, Conv2d: 5, BatchNorm2d: 5, ReLU: 4, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Bottleneck: 3, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, BatchNorm2d: 4, ReLU: 4, Conv2d: 4, Batc

In [ ]:
import torch
import torchinfo
from torchvision.models import resnet50
from model.level_1_classifier import level_1_classifier, level_1_output_layer, level_1_model
from jbu_module import JointBilateralUpsampling  # Ensure you import the JBU module

# Fine-tune and load model
def fine_tune_load_model():
    model = resnet50(weights="IMAGENET1K_V2")
    modules = list(model.children())[:-1]
    new_model = torch.nn.Sequential(*modules)
    new_model.fc = torch.nn.Flatten()
    return new_model

FE_model = fine_tune_load_model()

# Directory and training parameters
used_model_feature_size = 2048  
feature_size_extract_from_level_1 = 64

# Define the new classifier with JBU
classifier_level_1_with_jbu = level_1_classifier(feature_size=used_model_feature_size, feature_size_extract_from_level_1=feature_size_extract_from_level_1)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1=feature_size_extract_from_level_1, no_class=2)
model_level_1 = level_1_model(FE_model, classifier_level_1_with_jbu, output_layer_level_1)

# Create dummy inputs for summary
dummy_input = torch.randn(1, 3, 224, 224)  # Batch size of 1 for simplicity
dummy_guidance = torch.randn(1, 3, 224, 224)  # Same size as the input image

# Use torchinfo to summarize the model with both inputs
torchinfo.summary(model_level_1, input_data=(dummy_input, dummy_guidance), col_names=('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose=1)


# Training and teseting Level 1

In [ ]:
trainer = Trainer_level_1(model = model_level_1, level=1, training_dataloader = train_dataloader, validation_dataloader = val_dataloader, epoch = no_epoch, learning_rate = lr, use_gpu = True, opt_method = optimizer, checkpoint_dir = level_1_checkpoint_dir, checkpoint_filename=level_1_model_file, csv_logger = level_1_csvlogger_file  )

In [ ]:
trainer.run()

In [ ]:
# from config.Tester import Tester
tester = Tester_level_1(model = model_level_1, weights_file = level_1_model_file, level =1,  data_loader = test_dataloader, checkpoint_dir = level_1_checkpoint_dir)

In [ ]:
tester.prediction()

# Run Garbage Collector

In [ ]:
import gc
# del train_dataloader
# del val_dataloader
# del test_dataloader
del classifier_level_1
del output_layer_level_1
del model_level_1
del trainer
del tester
gc.collect()

# Level 2 Parameters

In [ ]:
no_epoch = 100
lr = 0.001
optimizer = 'AdamW'
level_2_checkpoint_dir = './checkpoints/'+used_model+'/level_2_'
fiction_gamma_pos = 4
fiction_gamma_neg = 5
non_fiction_gamma_pos = 4
non_fiction_gamma_neg = 5

# Feature Extraction for level 2

In [ ]:
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size,feature_size_extract_from_level_1=feature_size_extract_from_level_1, model=None)
# level_1_model = assign_gpu_model(level_1_model)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1 = feature_size_extract_from_level_1, no_class = 2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)
model_level_1.load_state_dict(torch.load(level_1_weights_path))

In [ ]:
def freeze_used_model(model):
    # model = resnet50(weights="IMAGENET1K_V2")
    print(model.modules)
    # print(model.fc.in_features)
    
    modules=list(model.children())[:-2]
    # print(modules)
    new_model= torch.nn.Sequential(*modules)
    for p in new_model.parameters():
        p.requires_grad = False
    # new_model.fc = torch.nn.Flatten()

    return new_model

def freeze_level_1_model(model):
    # model = resnet50(weights="IMAGENET1K_V2")
    print(model.modules)
    # print(model.fc.in_features)
    
    modules=list(model.children())[:-1]
    # print(modules)
    new_model= torch.nn.Sequential(*modules)
    for p in new_model.parameters():
        p.requires_grad = False
    # new_model.fc = torch.nn.Flatten()

    return new_model

def freeze_full_model(model):
    # model = resnet50(weights="IMAGENET1K_V2")
    print(model.modules)
    # print(model.fc.in_features)
    
    modules=list(model.children())
    # print(modules)
    new_model= torch.nn.Sequential(*modules)
    for p in new_model.parameters():
        p.requires_grad = False
    # new_model.fc = torch.nn.Flatten()

    return new_model

In [ ]:
used_model = freeze_used_model(copy.deepcopy(model_level_1))

In [ ]:
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size,feature_size_extract_from_level_1=feature_size_extract_from_level_1, model=None)
# level_1_model = assign_gpu_model(level_1_model)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1 = feature_size_extract_from_level_1, no_class = 2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)
model_level_1.load_state_dict(torch.load(level_1_weights_path))

FE_model_level_1 = freeze_level_1_model(copy.deepcopy(model_level_1))

In [ ]:
classifier_level_1 = level_1_classifier(feature_size=used_model_feature_size,feature_size_extract_from_level_1=feature_size_extract_from_level_1, model=None)
# level_1_model = assign_gpu_model(level_1_model)
output_layer_level_1 = level_1_output_layer(feature_size_extract_from_level_1 = feature_size_extract_from_level_1, no_class = 2)
model_level_1 = level_1_model(FE_model, classifier_level_1, output_layer_level_1)
model_level_1.load_state_dict(torch.load(level_1_weights_path))

model_level_1 = freeze_full_model(copy.deepcopy(model_level_1))

In [ ]:

torchinfo.summary(used_model, (3,224,224), batch_dim = 0, col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 1)

In [ ]:

torchinfo.summary(FE_model_level_1, (3,224,224), batch_dim = 0, col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 1)

In [ ]:

torchinfo.summary(model_level_1, (3,224,224), batch_dim = 0, col_names = ('input_size', 'output_size', 'num_params', 'kernel_size', 'mult_adds'), verbose = 1)

# Level 2 classifiers

In [ ]:
fiction_classifier = level_2_classifier(model_feature_size = used_model_feature_size, feature_from_level_1=feature_size_extract_from_level_1)
fiction_op_layer = level_2_output_layer(feature_size_extract_from_level_2=64, no_class = 29)
fiction_model = level_2_post_model(level_2_classifier = fiction_classifier, level_2_output_layer = fiction_op_layer)

In [ ]:
non_fiction_classifier = level_2_classifier(model_feature_size = used_model_feature_size, feature_from_level_1=feature_size_extract_from_level_1)
non_fiction_op_layer = level_2_output_layer(feature_size_extract_from_level_2=64, no_class = 29)
non_fiction_model = level_2_post_model(level_2_classifier = non_fiction_classifier, level_2_output_layer = non_fiction_op_layer)

In [ ]:
level_2_pre_model = level_2_pre_model_concate(used_model, FE_model_level_1)

In [ ]:
level_2_pre_model

# Train Test Level 2 Model

In [ ]:
level_2_trainer = Trainer_level_2(level_1_model = model_level_1, level_2_pre_model = level_2_pre_model, fiction_model =  fiction_model, non_fiction_model = non_fiction_model, training_dataloader = train_dataloader, validation_dataloader = val_dataloader, epoch = no_epoch, learning_rate = lr, use_gpu = True, opt_method = optimizer, fiction_checkpoint_dir = level_2_checkpoint_dir+'fiction', non_fiction_checkpoint_dir = level_2_checkpoint_dir+'non_fiction', checkpoint_filename = level_1_model_file, fiction_gamma_pos = fiction_gamma_pos, fiction_gamma_neg = fiction_gamma_neg, non_fiction_gamma_pos = non_fiction_gamma_pos, non_fiction_gamma_neg = non_fiction_gamma_neg, csv_logger = level_1_csvlogger_file)

In [ ]:
level_2_trainer.run()

In [ ]:
from config.Tester import Tester_level_2

In [ ]:
tester_level_2 = Tester_level_2(level_1_model = model_level_1, level_2_pre_model = level_2_pre_model, fiction_model =  fiction_model, non_fiction_model = non_fiction_model, weights_file = level_1_model_file, data_loader = test_dataloader, fiction_checkpoint_dir = level_2_checkpoint_dir+'fiction', non_fiction_checkpoint_dir = level_2_checkpoint_dir+'non_fiction', fiction_gamma_pos = fiction_gamma_pos, fiction_gamma_neg = fiction_gamma_neg, non_fiction_gamma_pos = non_fiction_gamma_pos, non_fiction_gamma_neg = non_fiction_gamma_neg)

In [ ]:
tester_level_2.prediction()